# lab2_2

In [126]:
import math
import numpy as np
import sympy as sm
from sympy import Derivative, diff
from itertools import groupby
import matplotlib.pyplot as plt
from tabulate import tabulate
from pprint import pprint

from IPython.display import display, Latex, clear_output

In [127]:
t, y =sm.symbols('t, y')
x=sm.Function('x')
p=sm.Function('p')

In [128]:
sm.init_printing(use_latex='mathjax')
x, y, t = sm.symbols('x, y, t', real=True)
C1, C2 = sm.symbols("C1, C2", real=True)
c = sm.symbols("c")
u = sm.Function('u')
ux = u(x)
ux1 = Derivative(u(x), x)
ux2 = Derivative(u(x), (x, 2))

h, uup1, uum1, uu0 = sm.symbols('h, u_i+1, u_i-1, u_i')

uup1, uum1, uu0

(uᵢ₊₁, uᵢ₋₁, uᵢ)

In [129]:
zad1 = {
    'a': 0, 'b': 2,
    'k': 0.5, 'T': 0.4,
    'fx': sm.lambdify(x, 1), 'fxt': 2,
    'g1t': sm.lambdify(t, sm.exp(-t)), 'g2t': sm.lambdify(t, sm.exp(-5*t))
    
}

In [154]:
n = 9
m = 20
hh = (zad1['b'] - zad1['a']) / (n-1)
tt = zad1['T'] / (m-1)
bb = zad1['k'] * tt / hh**2
uut1 = (uu0 - y)/(t)
uux2 = (uup1 - 2*uu0 + uum1)/(h**2)
apro = sm.Eq(uut1, zad1['k']*uux2 + zad1['fxt'])

us = sm.symbols(' '.join([f"u{i}" for i in range(n)]))
list_x = [zad1['a'] + hh*i for i in range(n)]
list_tt = [tt*i for i in range(m)]
list_g1t = [zad1['g1t'](list_tt[i]) for i in range(m)]
list_g2t = [zad1['g2t'](list_tt[i]) for i in range(m)]

print(hh, tt)
display(uux1, uux2)
display(apro)
print(us)
#print("list_x:", *list_x)
#print("list_tt:", *list_tt)
#print("list_g1t:", *list_g1t)
#print("list_g2t:", *list_g2t)

0.25 0.021052631578947368


8.0⋅uᵢ₊₁ - 8.0⋅uᵢ₋₁

-2⋅uᵢ + uᵢ₊₁ + uᵢ₋₁
───────────────────
          2        
         h         

uᵢ - y       0.5⋅(-2⋅uᵢ + uᵢ₊₁ + uᵢ₋₁)
────── = 2 + ─────────────────────────
  t                       2           
                         h            

(u0, u1, u2, u3, u4, u5, u6, u7, u8)


In [161]:
table = [[None]*(n+2) for _ in range(m+4)]
table[0][0] = ''
table[0][1] = 'i'
for i in range(2, n+2):
    table[0][i] = i-2
table[1][0] = 'j'
table[1][1] = 'tj \\ xi'
for i in range(2, n+2):
    table[1][i] = list_x[i-2]
    
for i in range(2, m+2):
    table[i][0] = i-2
for i in range(2, m+2):
    table[i][1] = list_tt[i-2]
table[m+4-2][1] = "й(х;%s)" % table[m+4-3][1]
table[m+4-1][1] = "|й - u|"
for i in range(m):
    table[i+2][2] = list_g1t[i]
    table[i+2][n+2-1] = list_g2t[i]
 
for i in range(1, n-1):
    table[2][i+2] = zad1['fx'](list_x[i])

for i in range(1, m):
    tyts = [apro.subs({x: table[1][2+j], y: table[2+i-1][2+j], h: hh, t: tt, uum1: us[j-1], uu0: us[j], uup1: us[j+1]})  for j in range(1,n-1)]
    tyts.append(sm.Eq(us[0], table[2+i][2]))
    tyts.append(sm.Eq(us[n-1], table[2+i][2+n-1]))
    #display(*tyts)
    result = list(sm.solve(tyts, us).values())
    #print()
    #display(*result)
    #break
    for j in range(1, n-1):
        table[2+i][2+j] = result[j]
        
print(tabulate(table, floatfmt=".04", tablefmt='fancy_grid'))

╒════╤══════════════════════════╤════════╤═══════╤═══════╤═══════╤═══════╤═══════╤═══════╤════════╤════════╕
│    │ i                        │ 0.0    │ 1.0   │ 2.0   │ 3.0   │ 4.0   │ 5.0   │ 6.0   │ 7.0    │ 8.0    │
├────┼──────────────────────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┤
│ j  │ tj \ xi                  │ 0.0    │ 0.25  │ 0.5   │ 0.75  │ 1.0   │ 1.25  │ 1.5   │ 1.75   │ 2.0    │
├────┼──────────────────────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┤
│ 0  │ 0.0                      │ 1.0    │ 1.0   │ 1.0   │ 1.0   │ 1.0   │ 1.0   │ 1.0   │ 1.0    │ 1.0    │
├────┼──────────────────────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┤
│ 1  │ 0.021052631578947368     │ 0.9792 │ 1.034 │ 1.041 │ 1.042 │ 1.042 │ 1.042 │ 1.04  │ 1.024  │ 0.9001 │
├────┼──────────────────────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┤
│ 2  │ 0.0421052631

In [ ]:
hello = 'hello'